

1. **Original VGG Paper**:
   - [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)

2. **VGG16 PyTorch Documentation**:
   - [torchvision.models.vgg16](https://pytorch.org/docs/stable/generated/torchvision.models.vgg16.html)

3. **VGG16 Wikipedia Page**:
   - [VGG16 Wikipedia](https://en.wikipedia.org/wiki/VGG16)

4. **VGG16 TensorFlow Hub Documentation**:
   - [TensorFlow Hub - VGG16 Documentation](https://tfhub.dev/google/imagenet/vgg16/feature_vector/4)

5. **VGG16 Implementation in TensorFlow**:
   - [VGG16 Implementation in TensorFlow](https://github.com/tensorflow/models/blob/master/research/slim/nets/vgg.py)

![VGG16 Architecture](https://static.packt-cdn.com/products/9781838827069/graphics/assets/0c28bb91-62aa-4165-a1fe-210d6ab63859.png)


### Importing required libraries

In [1]:
!pip install wandb

In [2]:
import os
import wandb
os.environ["WANDB_NOTEBOOK_NAME"] = "PartB.ipynb"
wandb.login()
wandb.login(key='13718865007c1068066166d683d0ce0cf87ec304')

wandb: Currently logged in as: ge23m018. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ge22m009/.netrc


True

In [3]:
import torch
import torchvision 
from torchvision.datasets import FashionMNIST
from torchvision import datasets, models
import torchvision.transforms as transforms
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, ConcatDataset, random_split
import matplotlib.pyplot as plt
import numpy as np
import glob

### Importing the pretrained VGG Net16 model

In [4]:
vggnet = torchvision.models.vgg16(weights=True)

/home/ge22m009/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/ge22m009/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 246MB/s] 


In [5]:
vggnet

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### Loading the iNaturalist Dataset 

In [6]:
def load_dataset(data_augmentation , train_path, test_path, train_batch_size, val_batch_size, test_batch_size):
    # Define the transformation for the input images
    transformer1 = transforms.Compose([
        transforms.Resize((256, 256)),  # Resize images to 256x256 pixels
        transforms.ToTensor(),  # Convert images to PyTorch tensors
        transforms.Normalize(mean=[0.4602, 0.4495, 0.3800], std=[0.2040, 0.1984, 0.1921])  # Normalize images
    ])
    
    # Load the training dataset with the defined transformation
    train_Dataset = torchvision.datasets.ImageFolder(train_path, transform=transformer1)
    
    # Split the training dataset into training and validation sets
    train_datasize = int(0.8 * len(train_Dataset))
    train_Dataset, val_Dataset = random_split(train_Dataset, [train_datasize, len(train_Dataset) - train_datasize])
    
    # Apply data augmentation if specified
    if data_augmentation == True: 
        transformer2 = transforms.Compose([
            transforms.Resize((256, 256)),  # Resize images to 256x256 pixels
            transforms.RandomHorizontalFlip(0.5),  # Randomly flip images horizontally with a probability of 0.5
            transforms.RandomVerticalFlip(0.02),  # Randomly flip images vertically with a probability of 0.02
            transforms.RandomRotation(degrees=45),  # Randomly rotate images by up to 45 degrees
            transforms.ToTensor(),  # Convert images to PyTorch tensors
            transforms.Normalize(mean=[0.4602, 0.4495, 0.3800], std=[0.2040, 0.1984, 0.1921])  # Normalize images
        ])
        
        # Create an augmented dataset with the defined transformation
        augmented_dataset = torchvision.datasets.ImageFolder(train_path, transform=transformer2)
        augmented_dataset_size = int(0.2 * len(augmented_dataset))
        
        # Split the augmented dataset into training and validation sets
        augmented_dataset, _  =  random_split(augmented_dataset, [augmented_dataset_size, len(augmented_dataset) - augmented_dataset_size])
        
        # Concatenate the original training dataset with the augmented dataset
        train_Dataset = ConcatDataset([train_Dataset, augmented_dataset])
    
    # Create data loaders for the training, validation, and test sets
    train_Loader = DataLoader(
        train_Dataset, 
        batch_size=train_batch_size,  # Set the batch size for the training data loader
        shuffle=True)  # Shuffle the training data
    
    test_Loader = DataLoader(
        test_path,  # Provide the path to the test dataset
        batch_size=test_batch_size,  # Set the batch size for the test data loader
        shuffle=True)  # Shuffle the test data
    
    val_Loader = DataLoader(
        val_Dataset, 
        batch_size=val_batch_size,  # Set the batch size for the validation data loader
        shuffle=True)  # Shuffle the validation data
    
    return train_Loader, val_Loader, test_Loader

# Set the paths to the training and test datasets
train_path = '/home/ge22m009/inaturalist_12K/train/'  # Path to the training dataset
test_path = '/home/ge22m009/inaturalist_12K/val/'  # Path to the test dataset

# Define batch sizes for training, validation, and test data loaders
train_batch_size = 64
test_batch_size = 16
val_batch_size = 16

# Specify whether to apply data augmentation
is_Data_Augmentation = True

# Load the datasets and create data loaders
train_Loader, val_Loader, test_Loader = load_dataset(is_Data_Augmentation, train_path, test_path, train_batch_size, val_batch_size, test_batch_size)

# Print the lengths of the training, validation, and test data loaders
print(len(train_Loader), len(val_Loader), len(test_Loader))


157 125 3


### Setting device to 'cuda' if GPU is available.

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Freezing the Model Parameters

In [8]:
for param in vggnet.parameters():
    param.requires_grad = False

### Adding One more layer to the model.

![Colab Markdown](https://miro.medium.com/v2/resize:fit:827/1*UeAhoKM0kJfCPA03wt5H0A.png)


In [9]:
vggnet.classifier[6] = torch.nn.Linear(in_features = 4096, out_features = 10)
vggnet.classifier.add_module("7", torch.nn.LogSoftmax(dim=1))
vggnet

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [10]:
vggnet.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## Function to train the model which logs Train accuracy, Train loss, Validataion accuracy & Validation loss 

In [11]:
def train(model, learning_rate, epochs, train_Loader, val_Loader, train_count, test_count, is_wandb_log): 
    # Define the loss function
    loss_function = torch.nn.CrossEntropyLoss()
    # Define the optimizer
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay=1e-4)

    # Loop over the specified number of epochs
    for epoch in range(epochs):
        train_accuracy = 0
        train_loss = 0
        model.train()  # Set the model to training mode
        # Iterate over the training data
        for i, (images, labels) in enumerate(train_Loader):

            images, labels = images.to(device), labels.to(device)
            # Zero the gradients
            optimizer.zero_grad()

            # Forward propagation
            y_pred = model(images)

            # Calculate the loss
            loss = loss_function(y_pred, labels)

            # Backward propagation
            loss.backward()

            # Update parameters
            optimizer.step()

            # Accumulate the training loss
            train_loss += loss.item()

            # Calculate training accuracy
            _, prediction = torch.max(y_pred.data, 1)
            train_accuracy += int(torch.sum(prediction == labels.data))
    
        # Calculate average training accuracy and loss
        train_accuracy /= train_count
        train_loss /= train_count
        print(f"Epochs : {epoch+1} Train Accuracy : {train_accuracy} Train Loss {train_loss}")
    
        test_accuracy = 0
        test_loss = 0
        # Evaluate on validation data
        with torch.no_grad():
            model.eval()  # Set the model to evaluation mode
            # Iterate over the validation data
            for i, (images, labels) in enumerate(val_Loader):
                images, labels = images.to(device), labels.to(device)

                # Forward propagation
                y_pred = model(images)

                # Calculate the loss
                loss = loss_function(y_pred, labels)
                test_loss += loss.item()

                # Calculate validation accuracy
                _, predicted = torch.max(y_pred.data, 1)
                test_accuracy += int(torch.sum(predicted == labels.data))

            # Calculate average validation accuracy and loss
            test_accuracy /= test_count
            test_loss /= test_count

            print(f"Epochs : {epoch+1} Validation Accuracy : {test_accuracy} Validation Loss {test_loss}")
            # Log metrics if WandB logging is enabled
            if is_wandb_log:
                wandb.log({"train_accuracy": train_accuracy, "train_loss": train_loss, "val_accuracy": test_accuracy, "val_error": test_loss}) 


### Return the count of Training dataset & testing dataset

In [12]:
def get_train_test_count(train_path, test_path):
    train_count = len(glob.glob(train_path+'/**/*.jpg'))
    test_count = len(glob.glob(test_path+'/**/*.jpg'))
    print("Training dataset count : ", train_count)
    print("Validation dataset count", test_count)
    return train_count, test_count

In [13]:
train_count, test_count = get_train_test_count(train_path, test_path)

Training dataset count :  9999
Validation dataset count 2000


## Training the model

In [14]:
# Import the wandb library for experiment tracking
import wandb

# Set the learning rate and number of epochs
learning_rate = 0.001
epochs = 8

# Specify whether to log metrics with WandB
is_wandb_log = True

# Initialize a WandB run with the specified project
run = wandb.init(project='dl2_ge23m018')

# Train the model using the specified hyperparameters and data loaders
train(vggnet, learning_rate, epochs, train_Loader, val_Loader, train_count, test_count, is_wandb_log)


Epochs : 1 Train Accuracy : 0.6791679167916792 Train Loss 0.01493758851974675
Epochs : 1 Validation Accuracy : 0.7625 Validation Loss 0.043095723293721674
Epochs : 2 Train Accuracy : 0.7414741474147415 Train Loss 0.012013182802574195
Epochs : 2 Validation Accuracy : 0.778 Validation Loss 0.04071123447269201
Epochs : 3 Train Accuracy : 0.7476747674767477 Train Loss 0.011450672017918764
Epochs : 3 Validation Accuracy : 0.777 Validation Loss 0.04020792128145695
Epochs : 4 Train Accuracy : 0.7606760676067607 Train Loss 0.011006195582870436
Epochs : 4 Validation Accuracy : 0.786 Validation Loss 0.03981682596355677
Epochs : 5 Train Accuracy : 0.7641764176417641 Train Loss 0.010849340619033712
Epochs : 5 Validation Accuracy : 0.7855 Validation Loss 0.03944969605654478
Epochs : 6 Train Accuracy : 0.7724772477247724 Train Loss 0.010720708216055714
Epochs : 6 Validation Accuracy : 0.786 Validation Loss 0.03995777470618486
Epochs : 7 Train Accuracy : 0.7772777277727773 Train Loss 0.01011817338335